# Google Gemini + Gaugid Integration

This notebook demonstrates how to integrate Gaugid SDK with Google Gemini 3 to build a personalized chatbot.

In [ ]:
import os
import asyncio
import google.generativeai as genai
from gaugid import GaugidClient

# Set your API keys
os.environ["GOOGLE_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
async def load_user_context(client: GaugidClient, user_did: str) -> str:
    """Load user context from Gaugid profile."""
    profile = await client.get_profile(
        user_did=user_did,
        scopes=["a2p:preferences", "a2p:professional", "a2p:context", "a2p:interests"]
    )
    
    memories = profile.get("memories", {}).get("episodic", [])
    context_parts = []
    for memory in memories:
        category = memory.get("category", "general")
        content = memory.get("content", "")
        context_parts.append(f"- [{category}] {content}")
    
    return "\n".join(context_parts) if context_parts else "No user context available."

# Initialize clients
gaugid_client = GaugidClient(connection_token=os.environ["GAUGID_CONNECTION_TOKEN"])
user_did = "did:a2p:user:demo"

# Load user context
user_context = await load_user_context(gaugid_client, user_did)
print("User context loaded:")
print(user_context)

In [ ]:
# Configure Gemini with user context
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

system_instruction = f"""You are a helpful AI assistant.

USER PROFILE (from Gaugid):
{user_context}

Adapt your responses to match user preferences and expertise level."""

model = genai.GenerativeModel(
    model_name="gemini-3-flash-exp",
    system_instruction=system_instruction
)

# Start chat
chat = model.start_chat(history=[])

# Example conversation
user_input = "What are the best practices for async Python?"
response = chat.send_message(user_input)

print(f"User: {user_input}")
print(f"\nAssistant: {response.text}")